In [ ]:
!pip install qiskit==1.4.2
!pip install qiskit_machine_learning==0.8.2
!pip install qiskit_algorithms==0.3.0
!pip install openpyxl
!pip install XlsxWriter
!pip install pylatexenc
!pip install qiskit_ibm_runtime
!pip install qiskit_ibm_transpiler

In [ ]:
QiskitRuntimeService.save_acount()

In [ ]:

import numpy as np
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from qiskit_ibm_runtime import QiskitRuntimeService, Batch, SamplerV2 as Sampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit.result import marginal_counts


random_seed = 12345
algorithm_globals.random_seed = random_seed
feature_dimension = 4
n_repeats = 2
linear_version = True  # True para Setosa vs Versicolor (sep linear), False para Versicolor vs Virginica


iris = datasets.load_iris()
X = iris.data
y = iris.target

class_to_remove = 2 if linear_version else 0
mask = iris.target != class_to_remove
X_full = X[mask]
y_full = y[mask]

# Hacemos las clases binarias (-1, 1)
y_full = np.where(y_full == y_full.min(), -1, 1)

# Escalado de características
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X_full)

service = QiskitRuntimeService(name=" ")
backend = service.backend("ibm_sherbrooke")
print(f"✅ Backend seleccionado: {backend.name}")

def build_custom_feature_map(n_qubits):
    x = ParameterVector('x', n_qubits)
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(n_qubits):
        qc.h(i)
        qc.rz(x[i], i)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    return qc, x

n_qubits = X_scaled.shape[1]
feature_map, params = build_custom_feature_map(n_qubits)


def build_kernel_matrix_real_ibm(X, feature_map, params, backend, shots=512, batch_size=1000):
    n_samples = X.shape[0]
    zero_state = '0' * n_qubits

    # Transpilación
    transpiled_circuits = []
    for x in X:
        bound = feature_map.assign_parameters(dict(zip(params, x)))
        transpiled = transpile(bound, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
        transpiled_circuits.append(transpiled)


    composed_circuits = []
    for i in range(n_samples):
        for j in range(n_samples):
            composed = transpiled_circuits[i].compose(transpiled_circuits[j].inverse())
            composed.measure(range(n_qubits), range(n_qubits))
            composed = transpile(composed, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
            composed_circuits.append(composed)


    print("\n🎨 Primer circuito compuesto:")
    print(composed_circuits[0].draw(fold=120, idle_wires=False))


    fidelities = []
    with Batch(backend=backend, max_time="3m") as batch:
        sampler = Sampler(mode=batch)
        jobs = []
        for i in range(0, len(composed_circuits), batch_size):
            job = sampler.run(composed_circuits[i:i+batch_size], shots=shots)
            jobs.append(job)

        for job in jobs:
            result = job.result()
            for pub_result in result:
                data_bin = pub_result.data
                reg_name = list(data_bin.__dict__.keys())[0]
                counts = getattr(data_bin, reg_name).get_counts()
                marginal = marginal_counts(counts, indices=list(range(n_qubits)))  # ⚡ Marginalizamos
                fid = marginal.get(zero_state, 0) / shots
                fidelities.append(fid)

    kernel_matrix = np.array(fidelities).reshape(n_samples, n_samples)
    return kernel_matrix

def build_kernel_matrix_cross_real_ibm(X_test, X_train, feature_map, params, backend, shots=512, batch_size=1000):
    n_test = X_test.shape[0]
    n_train = X_train.shape[0]
    zero_state = '0' * n_qubits

    transpiled_test = []
    for x in X_test:
        bound = feature_map.assign_parameters(dict(zip(params, x)))
        transpiled = transpile(bound, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
        transpiled_test.append(transpiled)

    transpiled_train = []
    for x in X_train:
        bound = feature_map.assign_parameters(dict(zip(params, x)))
        transpiled = transpile(bound, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
        transpiled_train.append(transpiled)

    composed_circuits = []
    for test_circ in transpiled_test:
        for train_circ in transpiled_train:
            composed = test_circ.compose(train_circ.inverse())
            composed.measure(range(n_qubits), range(n_qubits))
            composed = transpile(composed, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
            composed_circuits.append(composed)

    fidelities = []
    with Batch(backend=backend, max_time="3m") as batch:
        sampler = Sampler(mode=batch)
        jobs = []
        for i in range(0, len(composed_circuits), batch_size):
            job = sampler.run(composed_circuits[i:i+batch_size], shots=shots)
            jobs.append(job)

        for job in jobs:
            result = job.result()
            for pub_result in result:
                data_bin = pub_result.data
                reg_name = list(data_bin.__dict__.keys())[0]
                counts = getattr(data_bin, reg_name).get_counts()
                marginal = marginal_counts(counts, indices=list(range(n_qubits)))
                fid = marginal.get(zero_state, 0) / shots
                fidelities.append(fid)

    return np.array(fidelities).reshape(n_test, n_train)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_full, train_size=30, test_size=10, random_state=random_seed)


print("\nEjecutando construcción de matriz de entrenamiento...")
K_train = build_kernel_matrix_real_ibm(X_train, feature_map, params, backend)

print("\nEjecutando construcción de matriz de test...")
K_test = build_kernel_matrix_cross_real_ibm(X_test, X_train, feature_map, params, backend)
print("\nMatriz de kernel de test:")
print(K_test)

# Entrenamiento y evaluación
model = SVC(kernel='precomputed')
model.fit(K_train, y_train)Ha
y_pred = model.predict(K_test)

print("\nReporte final:")
print(classification_report(y_test, y_pred, labels=[-1, 1]))


In [ ]:
import pickle
import seaborn as sns
# (Después de hacer la predicción y mostrar el reporte)

# ========================
# GUARDADO FINAL EN .PKL
# ========================

resultados = {
    "K_train": K_train,
    "K_test": K_test,
    "y_train": y_train,
    "y_test": y_test,
    "y_pred": y_pred,
    "modelo_svc": model
}

filename = f"resultados_kernel_cuantico_{'linear' if linear_version else 'nolinear'}.pkl"

with open(filename, 'wb') as f:
    pickle.dump(resultados, f)

print(f"\nResultados guardados exitosamente en {filename}")


In [ ]:
import pickle

# Cargamos tu fichero
with open('resultados_kernel_cuantico_nolinear.pkl', 'rb') as f:
    resultados = pickle.load(f)

# Extraemos las variables
K_train = resultados['K_train']
K_test = resultados['K_test']
y_train = resultados['y_train']
y_test = resultados['y_test']

print("✅ Datos cargados correctamente desde resultados_kernel_cuantico_nolinear.pkl")

from sklearn.svm import SVC
from qiskit_machine_learning.algorithms import QSVC

# Modelo clásico
model_svc = SVC(kernel='precomputed')
model_svc.fit(K_train, y_train)
y_pred_svc = model_svc.predict(K_test)

# Modelo cuántico QSVC
model_qsvc = QSVC(quantum_kernel=None)  # quantum_kernel=None porque ya le pasamos la matriz
model_qsvc.fit(K_train, y_train)
y_pred_qsvc = model_qsvc.predict(K_test)
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Reporte de resultados
print("\n📊 Reporte SVC clásico:")
print(classification_report(y_test, y_pred_svc, labels=[-1, 1]))

print("\n📊 Reporte QSVC:")
print(classification_report(y_test, y_pred_qsvc, labels=[-1, 1]))

# Matriz de confusión SVC
plt.figure(figsize=(8,4))
sns.heatmap(confusion_matrix(y_test, y_pred_svc), annot=True, fmt="d", cmap="Blues")
plt.title("🔵 Matriz de Confusión - SVC clásico")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

# Matriz de confusión QSVC
plt.figure(figsize=(8,4))
sns.heatmap(confusion_matrix(y_test, y_pred_qsvc), annot=True, fmt="d", cmap="Purples")
plt.title("🟣 Matriz de Confusión - QSVC cuántico")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()



<ipython-input-5-d24ad73f3977>:24: QiskitMachineLearningWarning: 'No quantum kernel is provided, SamplerV1 based quantum kernel will be used.'
  model_qsvc = QSVC(quantum_kernel=None)  # quantum_kernel=None porque ya le pasamos la matriz


✅ Datos cargados correctamente desde resultados_kernel_cuantico_nolinear.pkl
